In [ ]:
ok="""
         _          _                  _           _        
        /\ \       / /\               / /\        / /\      
       /  \ \     / /  \             / /  \      / /  \     
      / /\ \ \   / / /\ \           / / /\ \__  / / /\ \__  
     / / /\ \_\ / / /\ \ \         / / /\ \___\/ / /\ \___\ 
    / / /_/ / // / /  \ \ \        \ \ \ \/___/\ \ \ \/___/ 
   / / /__\/ // / /___/ /\ \        \ \ \       \ \ \       
  / / /_____// / /_____/ /\ \   _    \ \ \  _    \ \ \      
 / / /      / /_________/\ \ \ /_/\__/ / / /_/\__/ / /      
/ / /      / / /_       __\ \_\\ \/___/ /  \ \/___/ /       
\/_/       \_\___\     /____/_/ \_____\/    \_____\/        
                                                            
"""

# 1) Representing graphs in code

There are multiple ways to represent graphs in code. In class we covered adjacency matrices (https://people.eecs.berkeley.edu/~vazirani/algorithms/chap3.pdf#page=2) and adjacency lists (https://people.eecs.berkeley.edu/~vazirani/algorithms/chap3.pdf#page=3). In general for 170, we choose to use the adjacency list representation since it let's us quickly determine all of a given nodes neighbors.

In many programming problems, verticies are typically labelled $0$ through $n-1$ for convenience (recall that arrays and lists in most languages begin at index 0). This allows us to represent an adjacency list using a list of lists. For the first exercise, store a given undirected graph as an adjacency list. Your input will be an integer $n$ representing the number of vertices in a graph and a list containing tuples with two elements. Each tuple represents the two endpoints of an undirected edge. (Hint: try using a list of lists or using a dictionary where the values are lists)

In [ ]:
# using a list of lists
def make_adj_list(n, edge_list):
    """
    return an adjacency list for a graph with nodes labelled 0 through n-1 given a list of edges in the graph

    args:
    n: an integer representing the number of nodes in a graph
    edge_list: a list of tuples. Each tuple (u,v) represents an undirected edge.
    """
    adj_list = [list() for _ in range(n)]
    for edge in edge_list:
        adj_list[edge[0]].append(edge[1])
        adj_list[edge[1]].append(edge[0])

    return adj_list

## Verification

To check that your adjacency list construction is correct, we generate 100 random graphs using networkx (this is a library that sometimes gets used in the class project), and make sure the adjacency list contains all the edges in the graph.

In [ ]:
import networkx as nx
for i in range (100):
    n = 50
    random_graph = nx.gnp_random_graph(n,0.6)
    edge_list = random_graph.edges

    # autograder
    adj_list = make_adj_list(n,edge_list)

    # 0) make sure adj_list is a list of lists or a dictionary of lists
    is_dict = type(adj_list) is dict
    assert is_dict or type(adj_list) is list, "Please return a list of lists or a dictionary of lists"


    # 1) assert all vertices are present in the adjacency list
    for i in range(0,n):
        if(is_dict):
            assert adj_list.get(i) is not None, "vertex "+str(i)+" is not in your adjacency list!"
        else:
            assert len(adj_list) >= i, "vertex "+str(i)+" is not in your adjacency list!"

    # 2) assert all edges are present in the adjacency list
    for edge in edge_list:
        assert edge[1] in adj_list[edge[0]], "edge ("+str(edge[0])+"," +str(edge[1])+") is not in your adjacency list!"
        assert edge[0] in adj_list[edge[1]], "edge ("+str(edge[1])+"," +str(edge[0])+") is not in your adjacency list!"

    # 3) assert there are no edges present in the adjacency list that are not present in the graph
    for i in range(0, n):
        for j in adj_list[i]:
            assert (i,j) in edge_list

print(ok)

# 2) DFS

In class we showed how to use DFS to check if there exists a path between two nodes, topologically sort nodes, and find SCC's. In those algorithms, pre and post numbers were used.

Here we'll implement a variation of DFS to print out the path between two nodes $s$ and $t$. In many problems, we want to be able to find the actual path between two nodes, not just determine if it exists. We can achieve this by passing in an extra data structure to our dfs call which for each node $u$, tells us what node $v$ will be the previous node on some root to $u$ path. By keeping track of this `prev` array you can then reconstruct the $s$-$t$ path by stepping through the `prev` array starting at $t$. Note you will not need to implement calculating pre and post numbers for this exercise.

In [ ]:
def return_path(adj_list, s, t, search):
    """
    Returns a list of nodes starting with s and ending with t representing an s to t path if it exists. 
    The path will be found using the 'search' function. Returns an empty list otherwise.

    Input:
        - adj_list: an adjacency list 
        - s: an int representing the starting node
        - t: an int representing the destination node
        - search(adj_list, curr, visited, prev): a graph traversal algorithm
    """
    
    visited = [False] * len(adj_list)
    prev = [None] * len(adj_list)
    
    search(adj_list, s, visited, prev)
    path = []
    if prev[t] is not None:
        current = t
        while prev[current] != s:
            path.append(current)
            current = prev[current]
        path.append(current)
        path.append(s)
    
    return list(reversed(path))

def dfs(adj_list, curr, visited, prev):
    """
    implement DFS here, the staff solution uses a recursive version of dfs similar to the textbook, 
    but feel free to write an iterative version
    
    args:
    adj_list: an adjacency_list representing the graph
    start: an int representing the node from which to begin traversing
    visited: an array of booleans representing if a vertex has been visited
    prev: an array of ints representing the previous node on a path from start to the current node
    """
    visited[curr] = True
    for v in adj_list[curr]:
        if not visited[v]:
            prev[v] = curr
            dfs(adj_list, v, visited, prev)

## Verification

To check that a valid path is found, we again use networkx to generate random graphs and check that the path returned is valid.

In [ ]:
import networkx as nx
for i in range(20):
    n = 10
    random_graph = nx.gnp_random_graph(n, 0.15)
    edge_list = random_graph.edges
    adj_list = make_adj_list(n, edge_list)
    path = return_path(adj_list, 0, n - 1, dfs)
    if len(path) == 0:
        assert len(list(nx.all_simple_paths(random_graph, 0, n-1))) == 0
    else:
        assert nx.is_simple_path(random_graph, path)

print(ok)

# 3) BFS

Similar to the above implementation of DFS, here we'll implement BFS such that BFS returns the s-t path. Since the code which generates the path is the same, you only need to implement BFS to work with the `return_path` method.

In [ ]:
import queue

def bfs(adj_list, start, visited, prev):
    """
    implement BFS here which updates the prev array

    args:
    adj_list: an adjacency_list representing the graph
    start: an int representing the node from which to begin traversing
    visited: an array of booleans representing if a vertex has been visited
    prev: an array of ints representing the previous node on a path from start to the current node
    """
    q = queue.Queue()
    
    # hint q.put(a) and q.get() pushes an element to the queue and pops off and 
    # returns the top element from the queue respectively
    visited[start] = True
    q.put(start)
    while q.qsize() > 0:
        current = q.get()
        for v in adj_list[current]:
            if not visited[v]:
                visited[v] = True
                prev[v] = current
                q.put(v)

## Verification

To check that a valid path is found, we again use networkx to generate random graphs and check that the path returned is valid.

In [ ]:
import networkx as nx
for i in range(100):
    n = 10
    random_graph = nx.erdos_renyi_graph(n, 0.3)
    edge_list = random_graph.edges
    adj_list = make_adj_list(n, edge_list)
    path = return_path(adj_list, 0, n-1, bfs)
    if len(path) == 0:
        assert len(list(nx.all_simple_paths(random_graph, 0, n-1))) == 0
    else:
        assert len(nx.shortest_path(random_graph, 0, n-1)) == len(path), (path, edge_list)

print(ok)

# 4) Shortest vanishing paths

For this problem assume you are given an directed unweighted graph and two nodes $s$ and $t$, and you want to find the shortest $s$ to $t$ path. However, you're additionally also given a function which takes in an edge as input and returns a boolean representing if an edge is traversable. It returns true if you are allowed to traverse that edge in your path and false otherwise. Write an algorithm which finds the shortest $s$ to $t$ path given this function.

There are two approaches to solving this problem. The first approach is to modify the BFS function. You modify BFS to only traverse edges the function returns true on. Intuitively, this solution works because we are only considering the edges that we are allowed to traverse and finding the shortest path through those edges. However, solutions which modify the algorithm they use can be a lot harder to prove for correctness -- imagine modifying FFT for a problem and proving your modified FFT works! Thus, a lot of problems in CS 170 involve using an algorithm as a black box (ie not modifying the algorithm at all) and instead modifying the inputs or outputs of that algorithm to solve the problem. To prove correctness of our solution, we then only need to show we are providing the correct inputs or interpretting the output correctly, rather than proving that the entire algorithm is correct.

Thus for your solution, we require you to take the second approach, which is to modify the input and/or output of BFS, which we know finds the shortest path on an unweighted graph. Your solution must call the the `return_path()` you implemented above and may not modify that method

In [ ]:
def shortest_vanishing_path(adj_list, s, t, func):
    """
    implement your solution here 
    
    args:
    adj_list: an adjacency_list representing the graph
    s: an int representing the node to start traversing from
    t: an int representing the target node
    func(u, v): returns true iff you're allowed to traverse edge u,v
    """
    
    # modify adj_list here
    adj_list_new = list(range(len(adj_list)))
    for u in range(len(adj_list)):
        adj_list_new[u] = [v for v in adj_list[u] if func(u, v)]
    
    return return_path(adj_list_new, s, t, bfs)

# Verification

We will use without proof (since the proof would be quite long), the fact that the first approach to this problem which modifies BFS produces the correct answer. To verify the correctness of your solution, we will simply check that your solution produces the same answer as the modified BFS approach. 

In [ ]:
def func(i, j):
    return (i + j) % 2 == 1

def shortest_vanishing_path_bfs(adj_list, start, visited, prev):
    """
    implementation of the first solution where BFS is modified to only take edges (u,v) which func(u,v)
    returns true

    args:
    adj_list: an adjacency_list representing the graph
    start: an int representing the node from which to begin traversing
    visited: an array of booleans representing if a vertex has been visited
    prev: an array of ints representing the previous node on a path from start to the current node
    """
    q = queue.Queue()
    q.put([start, start])
    while not q.empty():
        [prev_node, curr] = q.get()
        if not visited[curr]:
            visited[curr] = True
            prev[curr] = prev_node
            for v in adj_list[curr]:
                if func(curr,v): 
                    q.put([curr,v])

In [ ]:
def validate_path(adj_list, func, path):
    for i in range(len(path) - 1):
        assert path[i + 1] in adj_list[path[i]]
        assert func(path[i], path[i + 1])
    
for i in range(100):
    n = 100
    random_graph = nx.erdos_renyi_graph(n,0.3)
    
    edge_list = random_graph.edges
    adj_list = make_adj_list(n, edge_list)
    
    path_student = shortest_vanishing_path(adj_list, 0, n-1, func)
    path_bfs = return_path(adj_list, 0, n-1, shortest_vanishing_path_bfs)
    if len(path) == 0:
        assert len(list(nx.all_simple_paths(random_graph, 0, n - 1))) == 0
    else:
        assert len(path_bfs) == len(path_student)
        validate_path(adj_list, func, path_student)
        validate_path(adj_list, func, path_bfs)

print(ok)